In [3]:
# Imports
from pathlib import Path
import pandas as pd
import json
import csv
import geopandas as gpd

In [4]:
# List of colors to use in the timeseries (now only three, if you have more then 3 plots in 1 csv, please add some colors here!)
colors = ["#0178BE", "#B72025", "#75bbfd"]

In [10]:
def xlsx2json(filename, result_folder, plot_type="timeseries"):
    # Converts a csv file into multiple json files. Each row represents
    # a timeseries to be saved to a seperate json file
    shortname = filename.stem
    filename = filename.resolve()
    print(shortname)


    template = 'line_chart_template'
    # To create a line chart, there is a template available
    with open(f'{template}.json') as json_data:
        template = json.load(json_data)

    dfs = []

    with open(filename, 'r') as csvfile:
        nrows = (len(csvfile.readlines()))
    with open(filename, 'r') as csvfile:
        # the delimiter depends on how your CSV seperates values
        csvReader = csv.reader(csvfile, delimiter=';')
        data = []
        for index, row in enumerate(csvReader):
            if (index == nrows - 1):
                data.append(row)
            # check if row is empty
            if (not any(row)) or (index == nrows - 1):
                column = data[0][1]
                column_names = data.pop(0)
                df = pd.DataFrame(data, columns=column_names)
                df = df.set_index(column)
                dfs.append(df)
                data = []
            else:
                data.append(row)

    template['data']['labels'] = list(dfs[0].columns.values[1:])
    for name in dfs[0].index:
        new_data = template.copy()
        empty_ds = new_data['data']['datasets'][0].copy()
        new_data['data']['datasets'] = []
        for index, df in enumerate(dfs):
            empty_ds = empty_ds.copy()
            label = df.columns[0]
            df = df.iloc[:, 1:]
            data = df.loc[name, :].values
            # TODO: this is due to inconsistency in 1000 seperators in the csv files..
            data = [float(str(d).replace('.', '')) for d in data]
            data = [float(str(d).replace(',', '.')) for d in data]
            empty_ds['data'] = list(data)
            empty_ds['label'] = f'{label} {name}'
            empty_ds['backgroundColor'] = colors[index]
            empty_ds['borderColor'] = colors[index]

            new_data['data']['datasets'].append(empty_ds)

        output_folder = f'{result_folder}/{plot_type}/{name}'
        Path(output_folder).mkdir(parents=True, exist_ok=True)

        with open(f'{output_folder}/{shortname}.json', 'w+') as outfile:
            json.dump(new_data, outfile)


def get_overview_graph_files(data_folder):
    folders = Path(data_folder).rglob("*")
    files = [f.resolve() for f in folders]
    return files


In [15]:
# Define the folder where the csv files are stored for the timeseries
data_folder = r'N:/Projects/1230000/1230409/B. Measurements and calculations/9. Site in Peru/dashboard/data/csv/timeseries'
result_folder = '../'

# First get an overview of all the files living in the input folder
data_overview = get_overview_graph_files(data_folder)

# Loop over the files and create the line charts for each file for each area (area = defined in the file)
for filename in data_overview:
    xlsx2json(filename, result_folder)

[WindowsPath('//storage002.directory.intra/winprojects/1230000/1230409/B. Measurements and calculations/9. Site in Peru/dashboard/data/csv/timeseries/Agriculture Demand hist.csv'), WindowsPath('//storage002.directory.intra/winprojects/1230000/1230409/B. Measurements and calculations/9. Site in Peru/dashboard/data/csv/timeseries/Availability Groundwater hist.csv'), WindowsPath('//storage002.directory.intra/winprojects/1230000/1230409/B. Measurements and calculations/9. Site in Peru/dashboard/data/csv/timeseries/Availability Surface Water hist.csv'), WindowsPath('//storage002.directory.intra/winprojects/1230000/1230409/B. Measurements and calculations/9. Site in Peru/dashboard/data/csv/timeseries/Domestic Demand hist.csv'), WindowsPath('//storage002.directory.intra/winprojects/1230000/1230409/B. Measurements and calculations/9. Site in Peru/dashboard/data/csv/timeseries/Future agriculture demand SSP2.csv'), WindowsPath('//storage002.directory.intra/winprojects/1230000/1230409/B. Measurem

In [5]:
import matplotlib as mpl
from matplotlib.colors import ListedColormap

# This code cell is meant to create the color scale for the risk maps
vmin = 0
vmax = 1
cmap = ListedColormap(['#2B663C', '#489557', '#7CBA6D', '#B1D678', '#DDED97', '#FAE096', '#F2B06E', '#E4744F', '#E4744F', '#971D2B'])

norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
c = cmap(norm(10),bytes=True)

def get_color(value):
    c = cmap(norm(value),bytes=True)
    return '#{:02x}{:02x}{:02x}'.format(c[0], c[1], c[2])

In [11]:
def shape2geojson(shapefile, datafile, result_folder, common_attr):
  # This functrion reads out the shapefile as a  geopandas frame and combines it with an csv (which is converted
  # by pandas into a dataframe) a common attribute is needed to know which dataframe column is supposed to be added 
  # to which polygon. If no column has the same name, reuse the 1th column
  shortname = datafile.stem
  filename = datafile.resolve()
  print(shortname)

  gdf = gpd.read_file(shapefile)
  gdf = gdf.to_crs("EPSG:4326")
  df = pd.read_csv(filename, delimiter=';')
  df = df.iloc[:, 1:]
  for column in df.columns[1:]:
      colors = [get_color(val) for val in df[column].values]
      df[f'c_{column}'] = colors
  df['popupHTML'] = ["The risk for {name} is {value}." for x in range(len(df))]
  df['color'] = ["#000" for x in range(len(df))]
  df['fillOpacity'] = [1 for x in range(len(df))]

  # TODO: now assuming the 1st column is the column to merge into the geojson
  df.rename( columns={df.columns[0]: common_attr}, inplace=True )
  df['name'] = df[common_attr].values

  for name in df[common_attr]:
      mergeddf  = gdf.merge(df, on=common_attr, how='left')
      shape_file = gpd.GeoDataFrame(mergeddf)

      output_folder = f'{result_folder}/maps/{name}'
      Path(output_folder).mkdir(parents=True, exist_ok=True)
      shape_file.to_file(f'{output_folder}/{shortname}.geojson', driver='GeoJSON')


In [12]:
# Define the folder where the csv files are stored for the map data
data_folder = r'N:/Projects/1230000/1230409/B. Measurements and calculations/9. Site in Peru/dashboard/data/csv/maps/'
result_folder = '../'
shapefile = r"D:/voorCindy/shp/Subcuencas_Chirilu_selected_wgs84_18s.shp"
common_attr = 'Nom_Senamh'

data_overview = get_overview_graph_files(data_folder)
for filename in data_overview:
    shape2geojson(shapefile, filename, result_folder, common_attr)

Water Gap Score fut sector SSP2


TypeError: type numpy.ndarray doesn't define __round__ method

In [229]:
# IGNORE 
# def csv2scatter(filename, result_folder):
#     # Converts a csv file into multiple json files. Each row represents
#     # a timeseries to be saved to a seperate json file
#     shortname = filename.stem
#     filename = filename.resolve()
#     print(shortname)

#     # To create a line chart, there is a template available
#     with open('scatter_datalabels_template.json') as json_data:
#         template = json.load(json_data)

#     dfs = []

#     with open(filename, 'r') as csvfile:
#         nrows = (len(csvfile.readlines()))
#     with open(filename, 'r') as csvfile:
#         # the delimiter depends on how your CSV seperates values
#         csvReader = csv.reader(csvfile, delimiter=';')
#         data = []
#         for index, row in enumerate(csvReader):
#             if (index == nrows - 1):
#                 data.append(row)
#             # check if row is empty
#             if (not any(row)) or (index == nrows - 1):
#                 column = data[0][1]
#                 column_names = data.pop(0)
#                 df = pd.DataFrame(data, columns=column_names)
#                 df = df.set_index(column)
#                 dfs.append(df)
#                 data = []
#             else:
#                 data.append(row)

#     template['data']['labels'] = list(dfs[0].columns.values[1:])
#     for name in dfs[0].index:
#         new_data = template.copy()
#         empty_ds = new_data['data']['datasets'][0].copy()
#         new_data['data']['datasets'] = []
#         for index, df in enumerate(dfs):
#             empty_ds = empty_ds.copy()
#             label = df.columns[0]
#             df = df.iloc[:, 1:]
#             data = df.loc[name, :].values
#             # TODO: this is due to inconsistency in 1000 seperators in the csv files..
#             data = [float(str(d).replace('.', '')) for d in data]
#             data = [float(str(d).replace(',', '.')) for d in data]
#             empty_ds['data'] = list(data)
#             empty_ds['label'] = f'{label} {name}'
#             empty_ds['backgroundColor'] = colors[index]
#             empty_ds['borderColor'] = colors[index]

#             new_data['data']['datasets'].append(empty_ds)

#         output_folder = f'{result_folder}/timeseries/{name}'
#         Path(output_folder).mkdir(parents=True, exist_ok=True)

#         with open(f'{output_folder}/{shortname}.json', 'w+') as outfile:
#             json.dump(new_data, outfile)

# # Define the folder where the csv files are stored for the map data
# data_folder = r'N:/Projects/1230000/1230409/B. Measurements and calculations/9. Site in Peru/dashboard/data/csv/maps/'
# result_folder = '../'
# shapefile = r"D:/voorCindy/shp/Subcuencas_Chirilu_selected_wgs84_18s.shp"
# common_attr = 'Nom_Senamh'

# data_overview = get_overview_graph_files(data_folder)
# for filename in data_overview:
#     csv2scatter(shapefile, filename, result_folder, common_attr)

Water Gap Score fut sector SSP2
Water Gap Score fut sector SSP3
Water Gap Score future SSP2
Water Gap Score future SSP3
Water Gap Score sector
Water Gap score
Water scarcity Index (WSI) SSP2
Water scarcity Index (WSI) SSP3
Water Scarcity Index
